In [1]:
import requests as rq
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from xml.dom.minidom import parseString
%matplotlib inline

# Gode spørgsmål om dansk politik
Hvem stemmer mest nej? (person, parti)
Hvem stemmer mest ja?  (person, parti)
Hvis vi køre PCA (eller MCA eller CorEx) på personer, kommer de så til at cluster efter parti? Er der nogen der hører til et andet sted?
Hvis vi gør det samme på parti niveau ser vi så vestre vs højre fløj?
Hvem er den mest/mindst parti loyale person?


## 4 stemmetyper:
    * 1: For
    * 2: Imod
    * 3: Fraværende
    * 4: Hverken for eller imod
    
## 4 afstemningstyper
    * 1: Endelig
    * 2: Udvalgsindstilling
    * 3: Forslag til vedtagelse
    * 4: Ændringsforslag
    
## 5 Mødetyper
    * 1: Møde i salen
    * 2: Udvalgsmøde
    * 3: Andet møde
    * 4: ConciergeMeeting
    * 5: Statsrevisormøde
    
## 12 Aktørtyper
    * 5: Person

In [ ]:
# politiker serie: id->navn
# parti serie: id->navn
# index: tid, column: politiker id, value: parti id


In [32]:
#get polikter+parti count
params = {"$inlinecount": "allpages", "$filter": "typeid eq 5 or typeid eq 4"}
r = rq.get('http://oda.ft.dk/api/Aktør', params=params)
totalcount = int(r.json()['odata.count'])
print(totalcount)

3698


In [33]:
#get all politikere + patier
parti = {}
politiker = {}
ccount = 0
moreLeft = True
while moreLeft:
    if ccount>totalcount:
        ccount = totalcount
        moreLeft = False

    params = {"$filter": "typeid eq 5 or typeid eq 4", "$skip": ccount}
    r = rq.get('http://oda.ft.dk/api/Aktør', params=params)
    for aktor in r.json()['value']:
        if aktor['typeid'] == 5:
            politiker[aktor['id']] = aktor['navn']
        else:
            parti[aktor['id']] = aktor['navn']
    
    ccount += 20


In [38]:
politikere = pd.Series(politiker)
partier = pd.Series(parti)
print(politikere.head())
print(partier.head())

5             Frank Aaen
12        Nicolai Wammen
13           Sara Olsvig
17    Christine Antorini
18       Alex Ahrendtsen
dtype: object
269    Radikale Venstre
405    Radikale Venstre
406    Radikale Venstre
415    Radikale Venstre
430    Dansk Folkeparti
dtype: object


In [5]:
#politikerParti dataframe index: tid, column: politiker id, value: parti id
#fordi en politiker kan skifte parti


In [79]:
for i in politikere.index:
    r = rq.get('http://oda.ft.dk/api/Aktør({})/FraAktørAktør'.format(i), params={"$inlinecount": "allpages"})
    totalcount = int(r.json()['odata.count'])
    
    ccount = 0
    moreLeft = True
    while moreLeft:
        if ccount>totalcount:
            ccount = totalcount
            moreLeft = False

        params = {"$skip": ccount}
        r = rq.get('http://oda.ft.dk/api/Aktør({})/FraAktørAktør'.format(i), params=params)
        for gruppe in r.json()['value']:
            gru_id = gruppe['tilaktørid']
            fra = gruppe['startdato']
            til = gruppe['slutdato']
            if gru_id in partier:
                print(partier[gru_id], fra, til)
    
        ccount += 20
        break
    break
    
    #for parti in r.json()['value']:
        #så skal der loopes igennem TilAktørAktør (eller FraAktørAktør) for at finde alle medlemmer
        #parti dataframen har tid som index og politiker id som column også parti som værdi
        #således når vi har en afstemning, så kan jeg kigge på politiker id column og tids index også få et parti
    #    pass
    



Enhedslisten 2005-02-08T00:00:00 2005-10-04T00:00:00
Enhedslisten 2005-10-04T00:00:00 2006-10-03T00:00:00
Enhedslisten 2006-10-03T00:00:00 2007-10-02T00:00:00
Enhedslisten 2007-10-02T00:00:00 2007-11-13T00:00:00
Enhedslisten 2007-11-13T00:00:00 2008-10-07T00:00:00
Enhedslisten 2008-10-07T00:00:00 2009-10-06T00:00:00
Enhedslisten 2009-10-06T00:00:00 2010-10-05T00:00:00
Enhedslisten 2010-10-05T00:00:00 2011-09-15T00:00:00
Enhedslisten 2011-09-15T00:00:00 2011-10-04T00:00:00
Enhedslisten 2011-10-04T00:00:00 2012-10-02T00:00:00
Enhedslisten 2012-10-02T00:00:00 2013-10-01T00:00:00
Enhedslisten 2013-10-01T00:00:00 2014-10-07T00:00:00


In [ ]:
params = {"$filter":"typeid eq 1", "$expand": "Stemme"}
r = rq.get('http://oda.ft.dk/api/Afstemning', params=params)
r.json()

In [55]:
bla = pd.Series()

In [53]:
bla.loc[] = 3

In [72]:
a = pd.Series(index= pd.date_range("1/1/2011", "1/4/2011"), data = 3)
b = pd.Series(index= pd.date_range("1/3/2011", "1/11/2011"), data = 6)
c = pd.concat([a,b])
c['2011/01/01':'2011/01/05']

2011-01-01    3
2011-01-02    3
2011-01-03    3
2011-01-04    3
2011-01-03    6
2011-01-04    6
2011-01-05    6
dtype: int64

In [62]:
pd.DataFrame({'a':a,'b':b})

,a,b
2011-01-01,3.0,NaN
2011-01-02,3.0,NaN
2011-01-03,3.0,NaN
2011-01-04,3.0,NaN
2011-01-07,NaN,6.0
2011-01-08,NaN,6.0
2011-01-09,NaN,6.0
2011-01-10,NaN,6.0
2011-01-11,NaN,6.0
